# Import Assignments From a CSV File¶
In this example, a CSV file containing the locations of potholes will be imported into a Workforce Project as new assignments.

### Import ArcGIS API for Python
Import the `arcgis` library and some modules within it.

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce
from arcgis.geocoding import geocode

/Users/nira9586/anaconda3/envs/workforce-scripts/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,



### Connect to Organization And Get The Project
Let's connect to ArcGIS Online and find the new Project to add assignments to.

In [2]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)

Enter password: ········



### Load the CSV File¶
Let's use the pandas library to read the CSV file and display the potholes.

In [3]:
df = pd.read_csv("assignments.csv")
df

,Location,Description,Priority,Work Order Id,Due Date
0,480 E Tahquitz Canyon Way Palm Springs,Fill in the pothole,high,1,3/13/2019 22:43:00
1,456 E Andreas Rd Palm Springs,Fill in the pothole,medium,2,3/13/2019 21:43:05
2,200 E Arenas Rd Palm Springs,Fill in the pothole,low,3,3/13/2019 19:43:09
3,Skyline Trailhead Palm Springs,Fill in the pothole,critical,4,3/14/2019 10:43:00
4,Ruth Hardy Park Palm Springs,Fill in the pothole,high,5,3/14/2019 18:43:09
5,196 N Sunrise Way Palm Springs,Fill in the pothole,low,6,3/15/2019 14:00:00


### Create An Assignment For Each Row
For each assignment, First geocode the address to get the x,y location in (WGS84 Web Mercator) of the assignment. Then supply additional attributes.

Finally use the batch_add method to add multiple assignments at once (this is faster than using the add method since validation is performed once for all assignments).

In [4]:
assignments = []
for index, row in df.iterrows():
    geometry = geocode(f"{row['Location']}", out_sr=3857)[0]["location"]
    assignments.append(
        workforce.Assignment(
            project,
            geometry=geometry,
            location=row["Location"],
            description=row["Description"],
            priority=row["Priority"],
            work_order_id=row["Work Order Id"],
            assignment_type="Fill in Pothole",
            status="unassigned"
        )
    )
project.assignments.batch_add(assignments)

[<Assignment 122>,
 <Assignment 123>,
 <Assignment 124>,
 <Assignment 125>,
 <Assignment 126>,
 <Assignment 127>]


### Verify the assignments on the map
Let's verify that the assignments were created.

In [5]:
webmap = gis.map("Palm Springs", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap

MapView(layout=Layout(height='400px', width='100%'), zoom=14.0)